In [1]:
import os
import numpy as np
import cv2
from PIL import Image

In [2]:
Root = os.getcwd()
os.makedirs('Images_Folder', exist_ok=True)
os.makedirs('Result_Folder', exist_ok=True)

In [14]:
class SVD_Image_Compression_Proccessing:
    def __init__(self, Image_Name, Matrix_Approximation):
        self.Original_Matrix = self.Convert_Image_To_Matrix(Image_Name)
        self.K = Matrix_Approximation
        if self.Original_Matrix is not False:
            self.Shape = np.shape(self.Original_Matrix)
            Result = self.Singular_Value_Decomposition()
            self.Show_Image(Result)
        else:
            print(f"Picture not found!!!!")


    def Convert_to_PNG(self):
        for Image_File in os.listdir("Images_Folder"):
            if not Image_File.endswith(".png"):
                New_Image_File = os.path.splitext(Image_File)[0] + ".png"
                os.rename(
                    os.path.join("Images_Folder", Image_File),
                    os.path.join("Images_Folder", New_Image_File),
                )

    def Convert_Image_To_Matrix(self, Image_Name):
        self.Convert_to_PNG() 
        if Image_Name in os.listdir('Images_Folder'):
            print("Found the picture:", Image_Name)
            Image = cv2.imread(os.path.join('Images_Folder', Image_Name))
            Gray_Image = np.zeros((Image.shape[0], Image.shape[1]))
            for Row in range(Image.shape[0]):
                for Col in range(Image.shape[1]):
                    Pixel = Image[Row, Col]
                    Gray_Pixel = Pixel[0] * 0.114 + Pixel[1] * 0.587 + Pixel[2] * 0.299
                    Normalization = Gray_Pixel / 255
                    Gray_Image[Row, Col] = Normalization
            return Gray_Image
        else:
            return False
    
    def Find_Eigenvalues_and_Eigenvectors(self, Matrix):
        Eigenvalues, Eigenvectors = np.linalg.eig(Matrix)
        return Eigenvalues, Eigenvectors

    def Sigma_Matrix(self, Matrix):
        Singular_Values = np.sqrt(np.abs(self.Find_Eigenvalues_and_Eigenvectors(Matrix)[0]))
        Sigma_Matrix = np.zeros(self.Shape, dtype='float_')
        for i in range(min(len(Singular_Values), self.Shape[0], self.Shape[1])):
            Sigma_Matrix[i, i] = Singular_Values[i]
        return Sigma_Matrix
    
    def Singular_Value_Decomposition(self):
        A = self.Original_Matrix
        AtA = np.matmul(A.T, A)
        U = np.zeros((self.Shape[0], self.K), dtype='float_')
        D = self.Sigma_Matrix(AtA)
        V = self.Find_Eigenvalues_and_Eigenvectors(AtA)[1]
        
        for i in range(self.K):
            U[:, i] = np.matmul(A, V[:, i]) / D[i, i]
        Result = np.matmul(U[:, :self.K], D[:self.K, :self.K]) @ V[:, :self.K].T
        
        return Result

    def Show_Image(self, Result):
        Img_Array = np.clip(Result * 255, 0, 255).astype(np.uint8)
        if not os.path.exists('Result_Folder'):
            os.makedirs('Result_Folder')
        Filename = f"{self.K}_SVD_Image_Compression.jpg"
        cv2.imwrite(os.path.join('Result_Folder', Filename), Img_Array)

        Img = Image.fromarray(Img_Array)
        Img.show()

if __name__ == '__main__':
    SVD_Image_Compression_Proccessing('Cute Cat.png', 10)


Found the picture: Cute Cat.png
